In [384]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import random
from urllib.request import urlretrieve
from sklearn.linear_model import LinearRegression

%matplotlib inline

np.set_printoptions(suppress=True, formatter={'float': '{: 0.6f}'.format})

%reload_ext version_information
%version_information numpy, pandas, seaborn, urllib.request, random, sklearn

Software versions
Python 3.7.4 64bit [GCC 7.3.0]
IPython 7.8.0
OS Linux 4.15.0 66 generic x86_64 with debian buster sid
numpy 1.17.2
pandas 0.25.1
seaborn 0.9.0
urllib.request 3.7
random The 'random' distribution was not found and is required by the application
sklearn 0.21.3
Tue Nov 12 16:20:04 2019 CET

In [385]:
url = 'http://www-home.htwg-konstanz.de/~oduerr/data/fishing.npz'
npz_file_name = "../data/" + 'fishing.npz'

if not os.path.isfile(npz_file_name):
    print("Downloading")
    urlretrieve('http://www-home.htwg-konstanz.de/~oduerr/data/fishing.npz', npz_file_name)
else:
    print("File already downloaded.")
    d = np.load(npz_file_name)

File already downloaded.


In [386]:
header = ["livebait", "camper", "person", "children"]
train_data = np.asarray(d['Xt'])
test_data = np.asarray(d['Xte'])
train_labels = np.asarray(d['yt'])
test_labels = np.asarray(d['yte'])

In [392]:
%%latex
\begin{align}
Estimate: & \quad \mathbf{\hat{y}} && = \langle \mathbf{\hat{w}}, \mathbf{x} \rangle + \hat{w}_0 \\
Weights: & \quad \mathbf{\hat{w}} && = (X^T X)^{-1} X^T \mathbf{y}  \\
Intercept: & \quad \hat{w}_0 && = \bar{y} - \langle \mathbf{w}, \bar{\mathbf{x}} \rangle \\
\end{align}

<IPython.core.display.Latex object>

### Lineare Regression wie in Vorlesung

In [388]:
_w = np.linalg.inv(train_data.T @ train_data) @ train_data.T @ train_labels
_intercept = np.mean(train_labels) - _w @ np.mean(train_data, axis=0)
_y = np.sum(w.T * train_data, axis=1) + _intercept
mse = np.sum((train_labels - _y)**2) / 200
rmse = np.sqrt(mse)
print("Weights: {}, Intercept: {}, RMSE: {}".format(_w, _intercept, rmse))

Weights: [-1.699983  1.619098  3.162473 -5.049979], Intercept: -0.6354212760925293, RMSE: 11.05720786198419


### Lineare Regression mit sklearn

In [389]:
reg = LinearRegression().fit(train_data, train_labels)
_w = reg.coef_
_intercept = reg.intercept_
_y = np.sum(_w.T * train_data, axis=1) + _intercept
mse = np.sum((train_labels - _y)**2) / 200
rmse = np.sqrt(mse)
print("Weights: {}, Intercept: {}, RMSE: {}".format(_w, _intercept, rmse))

Weights: [ 2.482213  2.954309  4.649542 -5.471604], Intercept: -8.492233276367188, RMSE: 10.829104843834045
